In [2]:
import zipline
from zipline.api import order_target, record, symbol, symbols, order_target_percent, schedule_function, date_rules, set_benchmark, set_cancel_policy

from pykalman import KalmanFilter

import random
random.seed(123)

from datetime import datetime
import pytz

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '/Users/cemalarican/Desktop/THESIS/')
from pipeLine import Pipeline

years = [2015, 2016, 2017, 2018]
bands = [0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]
windows = [15, 30, 45, 60, 90, 120, 252]

file_path = '/Users/cemalarican/Desktop/THESIS/PART 3'



In [3]:
coint_pairs = pd.read_csv('/Users/cemalarican/Desktop/THESIS/PART 1/coint_pairs_to_investigate.csv')
coint_pairs

,stock1,stock2,constant,coeff,stat,p-value
0,MZZ,SMDD,1.721706e+01,0.653397,NON-STAT,6.567685e-02
1,EDV,UBT,1.901886e+01,1.156738,NON-STAT,1.173429e-01
2,IVV,IWB,-8.024626e+00,1.863519,STAT,2.437148e-02
3,DGL,SGOL,-3.365232e+00,3.555141,STAT,2.326552e-02
4,PHDG,VQT,-7.188207e-01,0.170467,STAT,5.418133e-07
5,IWV,SCHB,4.985746e+00,2.350143,NON-STAT,1.634537e-01
6,IWB,SPY,4.051654e+00,0.538618,STAT,1.627052e-02
7,IWB,SSO,4.734292e+01,0.912445,STAT,1.948852e-03
8,CURE,RXL,-1.056283e+01,0.340761,STAT,7.737721e-03
9,EUSA,VV,3.657650e-01,0.467233,STAT,1.498560e-13


In [7]:
## YEAR:

for year in [2018]:
    print('year:{}'.format(year))
    main_data = pd.DataFrame()
    pairs_sim = pd.DataFrame()
    
    for p in range(len(coint_pairs)):
        
        partial_main_data = pd.DataFrame()
        print(p)
        
        s1 = coint_pairs.iloc[p][0]
        s2 = coint_pairs.iloc[p][1]
        beta = round(coint_pairs.iloc[p][3],2)
        stat = coint_pairs.iloc[p][4]
        print(s1, s2, beta, stat)

        def initialize(context):
            context.pair = symbols(s1, s2)
            context.trading_day = 0

            # KEEPING TRACK IF WE GO LONG OR SHORT ON THE SPREAD
            context.long_on_spread = False
            context.shorting_on_spread = False

            context.entry1 = False
            context.entry2 = False
            context.stop = False
            context.exit = False

        def handle_data(context, data):
            context.trading_day += 1
            stock1 = context.pair[0]
            stock2 = context.pair[1]

            # determine weights:
            w2 = round(1/(1+ beta), 2)
            w1 = 1 - w2

            # use 7 previous days to get the filter up and running + plus the number of trading days.
            # daily update the filter:
            window = 7 + context.trading_day

            # SPREAD = stock1 - beta*stock2
            spread = data.history(stock1, 'close', window, '1d') - beta*data.history(stock2, 'close', window, '1d')
            
            spread_mean = spread.mean()
            spread_std = spread.std()
            current_spread = spread.iloc[-1]
            
            zscore = (current_spread - spread_mean)/spread_std

            # Apply Kalman Filter on Spread
            kf = KalmanFilter(transition_matrices=[1],
                          observation_matrices=[1],
                          initial_state_mean=0,
                          initial_state_covariance=1,
                          observation_covariance=1,
                          transition_covariance=.01)

            state_means, _ = kf.filter(spread.values)
            kalman_filter = pd.Series(state_means.flatten(), index=spread.index)

            kalman_mean = kalman_filter.mean()
            kalman_std = kalman_filter.std()
            current_kal = kalman_filter.iloc[-1]                       

            if kalman_std > 0:
                
                zscore_kalman = (current_kal - kalman_mean)/kalman_std
                
                # ENTRY CASE 1
                if zscore_kalman > 1.0 and not context.shorting_on_spread:
                    order_target_percent(stock1, -w1)
                    order_target_percent(stock2, w2)

                    context.shorting_spread = True
                    context.long_on_spread = False

                    context.entry1 = True
                    context.entry2 = False
                    context.stop = False
                    context.exit = False

                # ENTRY CASE 2:
                elif zscore_kalman < -1.0 and not context.long_on_spread:
                    order_target_percent(stock1, w1)
                    order_target_percent(stock2, -w2)
                    context.shorting_spread = False
                    context.long_on_spread = True

                    context.entry1 = False
                    context.entry2 = True
                    context.stop = False
                    context.exit = False

                # STOP POSITION    
                elif abs(zscore_kalman) >= 3:
                    order_target_percent(stock1, 0)
                    order_target_percent(stock2, 0)
                    context.shorting_spread = False
                    context.long_on_spread = False

                    context.entry1 = False
                    context.entry2 = False
                    context.stop = True
                    context.exit = False

                # EXIT STRATEGY - REACHING MEAN REVERSION
                elif abs(zscore_kalman) <= 0.05:
                    order_target_percent(stock1, 0)
                    order_target_percent(stock2, 0)
                    context.shorting_spread = False
                    context.long_on_spread = False

                    context.entry1 = False
                    context.entry2 = False
                    context.stop = False
                    context.exit = True

                record('zscore', zscore)  
                record('long_spread', context.long_on_spread )
                record('short_spread', context.shorting_on_spread )

                record('entry1', context.entry1)
                record('entry2', context.entry2)
                record('stop', context.stop)
                record('exit', context.exit)
                record('Z_KALMAN', zscore_kalman)

        start_time = datetime(2015, 1, 1, tzinfo=pytz.UTC)
        end_time = datetime(year+1, 1, 1, tzinfo=pytz.UTC)

        perf_Algo = zipline.run_algorithm(
                start=start_time,
                end=end_time,
                initialize=initialize,
                capital_base=100000,
                handle_data=handle_data,
                data_frequency='daily',
                bundle="eod-csvdir")

        pairs_sim[s1 + ' vs ' + s2 + ':' + stat] = perf_Algo.portfolio_value
        partial_main_data = perf_Algo

        partial_main_data['symbol'] = str(s1 + ' vs ' + s2 + ':' + stat)
        partial_main_data.index.rename('time', inplace=True)
        partial_main_data.set_index('symbol', append=True, inplace=True)

        main_data = pd.concat([main_data,partial_main_data])



#     print('finished year : {}'.format(year))
        print('dimension : {}'.format(main_data.shape))

        pairs_sim.to_csv(file_path + '/YEAR/pair_sim_'+ str(year) +'.csv') 
        main_data.to_csv(file_path + '/YEAR/MAIN_DATA/pair_sim_'+ str(year) +'.csv')
    file = open("/Users/cemalarican/Desktop/THESIS/PART 3/YEAR/" + str(year) + "_DONE" +".txt","w")
    file.close()


print('THE END')



year:2018
0
MZZ SMDD 0.65 NON-STAT


/opt/anaconda3/envs/zipline/lib/python3.5/site-packages/empyrical/stats.py:790: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
/opt/anaconda3/envs/zipline/lib/python3.5/site-packages/empyrical/stats.py:704: RuntimeWarning: invalid value encountered in true_divide
  out=out,


dimension : (1006, 45)
1
EDV UBT 1.16 NON-STAT
dimension : (2012, 45)
2
IVV IWB 1.86 STAT
dimension : (3018, 45)
3
DGL SGOL 3.56 STAT
dimension : (4024, 45)
4
PHDG VQT 0.17 STAT
dimension : (5030, 45)
5
IWV SCHB 2.35 NON-STAT
dimension : (6036, 45)
6
IWB SPY 0.54 STAT
dimension : (7042, 45)
7
IWB SSO 0.91 STAT
dimension : (8048, 45)
8
CURE RXL 0.34 STAT
dimension : (9054, 45)
9
EUSA VV 0.47 STAT
dimension : (10060, 45)
10
PHYS SGOL 0.87 NON-STAT
dimension : (11066, 45)
11
FXP YANG 0.31 NON-STAT
dimension : (12072, 45)
12
SCHB VTI 0.49 NON-STAT
dimension : (13078, 45)
13
DGL GLD 0.36 STAT
dimension : (14084, 45)
14
DLN VYM 1.07 NON-STAT
dimension : (15090, 45)
15
FUTY XLU 0.62 NON-STAT
dimension : (16096, 45)
16
FXY YCL 0.64 STAT
dimension : (17102, 45)
17
VPU XLU 2.09 NON-STAT
dimension : (18108, 45)
18
GLD PHYS 11.28 STAT
dimension : (19114, 45)
19
SCHG VOOG 0.53 STAT
dimension : (20120, 45)
20
SCHB VONE 0.52 NON-STAT
dimension : (21126, 45)
21
PGF PGX 1.16 STAT
dimension : (22132, 45

/opt/anaconda3/envs/zipline/lib/python3.5/site-packages/ipykernel/__main__.py:52: RuntimeWarning: invalid value encountered in double_scalars


dimension : (152912, 45)
152
GLD SGOL 9.81 STAT
dimension : (153918, 45)
153
BXE BXEFF 1.0 STAT
dimension : (154924, 45)
154
BXEFD BXEFF 1.0 STAT
dimension : (155930, 45)
155
TMV TTT 0.78 STAT
dimension : (156936, 45)
156
ECA OVV 0.2 STAT
dimension : (157942, 45)
157
SVXY XIV 3.92 STAT
dimension : (158948, 45)
158
CPRI KORS 1.0 STAT
dimension : (159954, 45)
159
AIM HEB 1.0 STAT
dimension : (160960, 45)
160
OXF WMLPQ 0.15 STAT
dimension : (161966, 45)
161
OXF WMLP 0.15 STAT
dimension : (162972, 45)
162
IVV VOO 1.09 STAT
dimension : (163978, 45)
163
IVV SPY 1.01 STAT
dimension : (164984, 45)
164
SPXL UPRO 0.95 STAT
dimension : (165990, 45)
165
SPY VOO 1.08 STAT
dimension : (166996, 45)
166
DF DFODQ 1.0 STAT
dimension : (168002, 45)
167
AAV AAVVF 1.0 STAT
dimension : (169008, 45)
168
ESV VAL 1.0 STAT
dimension : (170014, 45)
169
HCLP HCR 1.0 STAT
dimension : (171020, 45)
170
CRR CRRT 1.0 STAT
dimension : (172026, 45)
171
SUTR TOR 0.2 STAT
dimension : (173032, 45)
172
J JEC 1.0 STAT
dimens

In [9]:
## BANDS

for band in [1.75, 2]:
    print('BAND:{}'.format(band))
    main_data = pd.DataFrame()
    pairs_sim = pd.DataFrame()
    
    for p in range(len(coint_pairs)):
        print(p)
        
        partial_main_data = pd.DataFrame()
        
        s1 = coint_pairs.iloc[p][0]
        s2 = coint_pairs.iloc[p][1]
        beta = round(coint_pairs.iloc[p][3],2)
        stat = coint_pairs.iloc[p][4]
        print(s1, s2, beta, stat)

        def initialize(context):
            context.pair = symbols(s1, s2)
            context.trading_day = 0

            # KEEPING TRACK IF WE GO LONG OR SHORT ON THE SPREAD
            context.long_on_spread = False
            context.shorting_on_spread = False

            context.entry1 = False
            context.entry2 = False
            context.stop = False
            context.exit = False

        def handle_data(context, data):
            context.trading_day += 1
            stock1 = context.pair[0]
            stock2 = context.pair[1]

            # determine weights:
            w2 = round(1/(1+ beta), 2)
            w1 = 1 - w2

            # use 7 previous days to get the filter up and running + plus the number of trading days.
            # daily update the filter:
            window = 7 + context.trading_day

            # SPREAD = stock1 - beta*stock2
            spread = data.history(stock1, 'close', window, '1d') - beta*data.history(stock2, 'close', window, '1d')
            
            spread_mean = spread.mean()
            spread_std = spread.std()
            current_spread = spread.iloc[-1]
            
            zscore = (current_spread - spread_mean)/spread_std

            # Apply Kalman Filter on Spread
            kf = KalmanFilter(transition_matrices=[1],
                          observation_matrices=[1],
                          initial_state_mean=0,
                          initial_state_covariance=1,
                          observation_covariance=1,
                          transition_covariance=.01)

            state_means, _ = kf.filter(spread.values)
            kalman_filter = pd.Series(state_means.flatten(), index=spread.index)

            kalman_mean = kalman_filter.mean()
            kalman_std = kalman_filter.std()
            current_kal = kalman_filter.iloc[-1]

            if kalman_std > 0:
                
                zscore_kalman = (current_kal - kalman_mean)/kalman_std
                
                # ENTRY CASE 1
                if zscore_kalman > band and not context.shorting_on_spread:
                    order_target_percent(stock1, -w1)
                    order_target_percent(stock2, w2)

                    context.shorting_spread = True
                    context.long_on_spread = False

                    context.entry1 = True
                    context.entry2 = False
                    context.stop = False
                    context.exit = False

                # ENTRY CASE 2:
                elif zscore_kalman < -band and not context.long_on_spread:
                    order_target_percent(stock1, w1)
                    order_target_percent(stock2, -w2)
                    context.shorting_spread = False
                    context.long_on_spread = True

                    context.entry1 = False
                    context.entry2 = True
                    context.stop = False
                    context.exit = False

                # STOP POSITION    
                elif abs(zscore_kalman) >= 3:
                    order_target_percent(stock1, 0)
                    order_target_percent(stock2, 0)
                    context.shorting_spread = False
                    context.long_on_spread = False

                    context.entry1 = False
                    context.entry2 = False
                    context.stop = True
                    context.exit = False

                # EXIT STRATEGY - REACHING MEAN REVERSION
                elif abs(zscore_kalman) <= 0.05:
                    order_target_percent(stock1, 0)
                    order_target_percent(stock2, 0)
                    context.shorting_spread = False
                    context.long_on_spread = False

                    context.entry1 = False
                    context.entry2 = False
                    context.stop = False
                    context.exit = True

                record('zscore', zscore)  
                record('long_spread', context.long_on_spread )
                record('short_spread', context.shorting_on_spread )

                record('entry1', context.entry1)
                record('entry2', context.entry2)
                record('stop', context.stop)
                record('exit', context.exit)
                record('Z_KALMAN', zscore_kalman)

        start_time = datetime(2015, 1, 1, tzinfo=pytz.UTC)
        end_time = datetime(2016, 1, 1, tzinfo=pytz.UTC)

        perf_Algo = zipline.run_algorithm(
                start=start_time,
                end=end_time,
                initialize=initialize,
                capital_base=100000,
                handle_data=handle_data,
                data_frequency='daily',
                bundle="eod-csvdir")

        pairs_sim[s1 + ' vs ' + s2 + ':' + stat] = perf_Algo.portfolio_value
        partial_main_data = perf_Algo

        partial_main_data['symbol'] = str(s1 + ' vs ' + s2 + ':' + stat)
        partial_main_data.index.rename('time', inplace=True)
        partial_main_data.set_index('symbol', append=True, inplace=True)

        main_data = pd.concat([main_data,partial_main_data])



        print('dimension : {}'.format(main_data.shape))

        pairs_sim.to_csv(file_path + '/BANDS/pair_sim_'+ str(band) +'.csv') 
        main_data.to_csv(file_path + '/BANDS/MAIN_DATA/pair_sim_'+ str(band) +'.csv')
    print('finished band : {}'.format(band))

    file = open("/Users/cemalarican/Desktop/THESIS/PART 3/BANDS/" + str(band) + "_DONE" +".txt","w")
    file.close()


print('THE END')



BAND:1.75
0
MZZ SMDD 0.65 NON-STAT


/opt/anaconda3/envs/zipline/lib/python3.5/site-packages/empyrical/stats.py:704: RuntimeWarning: invalid value encountered in true_divide
  out=out,
/opt/anaconda3/envs/zipline/lib/python3.5/site-packages/empyrical/stats.py:790: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)


dimension : (252, 45)
1
EDV UBT 1.16 NON-STAT
dimension : (504, 45)
2
IVV IWB 1.86 STAT
dimension : (756, 45)
3
DGL SGOL 3.56 STAT
dimension : (1008, 45)
4
PHDG VQT 0.17 STAT
dimension : (1260, 45)
5
IWV SCHB 2.35 NON-STAT
dimension : (1512, 45)
6
IWB SPY 0.54 STAT
dimension : (1764, 45)
7
IWB SSO 0.91 STAT
dimension : (2016, 45)
8
CURE RXL 0.34 STAT
dimension : (2268, 45)
9
EUSA VV 0.47 STAT
dimension : (2520, 45)
10
PHYS SGOL 0.87 NON-STAT
dimension : (2772, 45)
11
FXP YANG 0.31 NON-STAT
dimension : (3024, 45)
12
SCHB VTI 0.49 NON-STAT
dimension : (3276, 45)
13
DGL GLD 0.36 STAT
dimension : (3528, 45)
14
DLN VYM 1.07 NON-STAT
dimension : (3780, 45)
15
FUTY XLU 0.62 NON-STAT
dimension : (4032, 45)
16
FXY YCL 0.64 STAT
dimension : (4284, 45)
17
VPU XLU 2.09 NON-STAT
dimension : (4536, 45)
18
GLD PHYS 11.28 STAT
dimension : (4788, 45)
19
SCHG VOOG 0.53 STAT
dimension : (5040, 45)
20
SCHB VONE 0.52 NON-STAT
dimension : (5292, 45)
21
PGF PGX 1.16 STAT
dimension : (5544, 45)
22
DOG DXD 0.4

/opt/anaconda3/envs/zipline/lib/python3.5/site-packages/ipykernel/__main__.py:52: RuntimeWarning: invalid value encountered in double_scalars


dimension : (38304, 45)
152
GLD SGOL 9.81 STAT
dimension : (38556, 45)
153
BXE BXEFF 1.0 STAT
dimension : (38808, 45)
154
BXEFD BXEFF 1.0 STAT
dimension : (39060, 45)
155
TMV TTT 0.78 STAT
dimension : (39312, 45)
156
ECA OVV 0.2 STAT
dimension : (39564, 45)
157
SVXY XIV 3.92 STAT
dimension : (39816, 45)
158
CPRI KORS 1.0 STAT
dimension : (40068, 45)
159
AIM HEB 1.0 STAT
dimension : (40320, 45)
160
OXF WMLPQ 0.15 STAT
dimension : (40572, 45)
161
OXF WMLP 0.15 STAT
dimension : (40824, 45)
162
IVV VOO 1.09 STAT
dimension : (41076, 45)
163
IVV SPY 1.01 STAT
dimension : (41328, 45)
164
SPXL UPRO 0.95 STAT
dimension : (41580, 45)
165
SPY VOO 1.08 STAT
dimension : (41832, 45)
166
DF DFODQ 1.0 STAT
dimension : (42084, 45)
167
AAV AAVVF 1.0 STAT
dimension : (42336, 45)
168
ESV VAL 1.0 STAT
dimension : (42588, 45)
169
HCLP HCR 1.0 STAT
dimension : (42840, 45)
170
CRR CRRT 1.0 STAT
dimension : (43092, 45)
171
SUTR TOR 0.2 STAT
dimension : (43344, 45)
172
J JEC 1.0 STAT
dimension : (43596, 45)
173

dimension : (21924, 45)
87
DXD SDOW 0.16 NON-STAT
dimension : (22176, 45)
88
SDS SPXU 0.34 NON-STAT
dimension : (22428, 45)
89
IWB IYY 1.13 STAT
dimension : (22680, 45)
90
QLD TQQQ 1.55 NON-STAT
dimension : (22932, 45)
91
ITOT IWB 0.41 STAT
dimension : (23184, 45)
92
DGL DGP 0.77 STAT
dimension : (23436, 45)
93
SPTL TLT 0.27 NON-STAT
dimension : (23688, 45)
94
IWB VV 1.19 STAT
dimension : (23940, 45)
95
ITOT VV 0.49 STAT
dimension : (24192, 45)
96
DBS SIVR 1.78 STAT
dimension : (24444, 45)
97
IVW SPYG 4.58 STAT
dimension : (24696, 45)
98
SPXL SSO 0.45 STAT
dimension : (24948, 45)
99
ITOT VONE 0.49 STAT
dimension : (25200, 45)
100
SDS SH 4.44 NON-STAT
dimension : (25452, 45)
101
VIXY VXX 0.84 STAT
dimension : (25704, 45)
102
VIXY VXX_ 0.84 STAT
dimension : (25956, 45)
103
IEV VGK 0.81 STAT
dimension : (26208, 45)
104
VIIX VXX 2.11 STAT
dimension : (26460, 45)
105
VIIX VXX_ 2.11 STAT
dimension : (26712, 45)
106
BBAR BFR 1.0 STAT
dimension : (26964, 45)
107
ITOT IYY 0.46 STAT
dimension : 

In [11]:
## MOVING WINDOWS

for moving_window in windows:
    print('BAND:{}'.format(moving_window))
    main_data = pd.DataFrame()
    pairs_sim = pd.DataFrame()
    
    for p in range(len(coint_pairs)):
        print(p)
        
        partial_main_data = pd.DataFrame()
        
        s1 = coint_pairs.iloc[p][0]
        s2 = coint_pairs.iloc[p][1]
        beta = round(coint_pairs.iloc[p][3],2)
        stat = coint_pairs.iloc[p][4]
        print(s1, s2, beta, stat)

        def initialize(context):
            context.pair = symbols(s1, s2)
            context.trading_day = 0

            # KEEPING TRACK IF WE GO LONG OR SHORT ON THE SPREAD
            context.long_on_spread = False
            context.shorting_on_spread = False

            context.entry1 = False
            context.entry2 = False
            context.stop = False
            context.exit = False

        def handle_data(context, data):
            context.trading_day += 1
            stock1 = context.pair[0]
            stock2 = context.pair[1]

            # determine weights:
            w2 = round(1/(1+ beta), 2)
            w1 = 1 - w2

            # use 7 previous days to get the filter up and running + plus the number of trading days.
            # daily update the filter:
            window = 7 + moving_window

            # SPREAD = stock1 - beta*stock2
            spread = data.history(stock1, 'close', window, '1d') - beta*data.history(stock2, 'close', window, '1d')
            
            spread_mean = spread.mean()
            spread_std = spread.std()
            current_spread = spread.iloc[-1]
            
            zscore = (current_spread - spread_mean)/spread_std

            # Apply Kalman Filter on Spread
            kf = KalmanFilter(transition_matrices=[1],
                          observation_matrices=[1],
                          initial_state_mean=0,
                          initial_state_covariance=1,
                          observation_covariance=1,
                          transition_covariance=.01)

            state_means, _ = kf.filter(spread.values)
            kalman_filter = pd.Series(state_means.flatten(), index=spread.index)

            kalman_mean = kalman_filter.mean()
            kalman_std = kalman_filter.std()
            current_kal = kalman_filter.iloc[-1]

            if kalman_std > 0:
                
                zscore_kalman = (current_kal - kalman_mean)/kalman_std
                
                # ENTRY CASE 1
                if zscore_kalman > 1.0 and not context.shorting_on_spread:
                    order_target_percent(stock1, -w1)
                    order_target_percent(stock2, w2)

                    context.shorting_spread = True
                    context.long_on_spread = False

                    context.entry1 = True
                    context.entry2 = False
                    context.stop = False
                    context.exit = False

                # ENTRY CASE 2:
                elif zscore_kalman < -1.0 and not context.long_on_spread:
                    order_target_percent(stock1, w1)
                    order_target_percent(stock2, -w2)
                    context.shorting_spread = False
                    context.long_on_spread = True

                    context.entry1 = False
                    context.entry2 = True
                    context.stop = False
                    context.exit = False

                # STOP POSITION    
                elif abs(zscore_kalman) >= 3:
                    order_target_percent(stock1, 0)
                    order_target_percent(stock2, 0)
                    context.shorting_spread = False
                    context.long_on_spread = False

                    context.entry1 = False
                    context.entry2 = False
                    context.stop = True
                    context.exit = False

                # EXIT STRATEGY - REACHING MEAN REVERSION
                elif abs(zscore_kalman) <= 0.05:
                    order_target_percent(stock1, 0)
                    order_target_percent(stock2, 0)
                    context.shorting_spread = False
                    context.long_on_spread = False

                    context.entry1 = False
                    context.entry2 = False
                    context.stop = False
                    context.exit = True

                record('zscore', zscore)  
                record('long_spread', context.long_on_spread )
                record('short_spread', context.shorting_on_spread )

                record('entry1', context.entry1)
                record('entry2', context.entry2)
                record('stop', context.stop)
                record('exit', context.exit)
                record('Z_KALMAN', zscore_kalman)

        start_time = datetime(2015, 1, 1, tzinfo=pytz.UTC)
        end_time = datetime(2016, 1, 1, tzinfo=pytz.UTC)

        perf_Algo = zipline.run_algorithm(
                start=start_time,
                end=end_time,
                initialize=initialize,
                capital_base=100000,
                handle_data=handle_data,
                data_frequency='daily',
                bundle="eod-csvdir")

        pairs_sim[s1 + ' vs ' + s2 + ':' + stat] = perf_Algo.portfolio_value
        partial_main_data = perf_Algo

        partial_main_data['symbol'] = str(s1 + ' vs ' + s2 + ':' + stat)
        partial_main_data.index.rename('time', inplace=True)
        partial_main_data.set_index('symbol', append=True, inplace=True)

        main_data = pd.concat([main_data,partial_main_data])



        print('finished band : {}'.format(band))
        print('dimension : {}'.format(main_data.shape))

        pairs_sim.to_csv(file_path + '/MOVING_WINDOW/pair_sim_'+ str(moving_window) +'.csv') 
        main_data.to_csv(file_path + '/MOVING_WINDOW/MAIN_DATA/pair_sim_'+ str(moving_window) +'.csv')
    file = open("/Users/cemalarican/Desktop/THESIS/PART 3/MOVING_WINDOW/" + str(moving_window) + "_DONE" +".txt","w")
    file.close()


print('THE END')



BAND:15
0
MZZ SMDD 0.65 NON-STAT


/opt/anaconda3/envs/zipline/lib/python3.5/site-packages/empyrical/stats.py:790: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
/opt/anaconda3/envs/zipline/lib/python3.5/site-packages/empyrical/stats.py:704: RuntimeWarning: invalid value encountered in true_divide
  out=out,


finished band : 2
dimension : (252, 45)
1
EDV UBT 1.16 NON-STAT
finished band : 2
dimension : (504, 45)
2
IVV IWB 1.86 STAT
finished band : 2
dimension : (756, 45)
3
DGL SGOL 3.56 STAT
finished band : 2
dimension : (1008, 45)
4
PHDG VQT 0.17 STAT
finished band : 2
dimension : (1260, 45)
5
IWV SCHB 2.35 NON-STAT
finished band : 2
dimension : (1512, 45)
6
IWB SPY 0.54 STAT
finished band : 2
dimension : (1764, 45)
7
IWB SSO 0.91 STAT
finished band : 2
dimension : (2016, 45)
8
CURE RXL 0.34 STAT
finished band : 2
dimension : (2268, 45)
9
EUSA VV 0.47 STAT
finished band : 2
dimension : (2520, 45)
10
PHYS SGOL 0.87 NON-STAT
finished band : 2
dimension : (2772, 45)
11
FXP YANG 0.31 NON-STAT
finished band : 2
dimension : (3024, 45)
12
SCHB VTI 0.49 NON-STAT
finished band : 2
dimension : (3276, 45)
13
DGL GLD 0.36 STAT
finished band : 2
dimension : (3528, 45)
14
DLN VYM 1.07 NON-STAT
finished band : 2
dimension : (3780, 45)
15
FUTY XLU 0.62 NON-STAT
finished band : 2
dimension : (4032, 45)
16
F

127
FENY IYE 0.52 STAT
finished band : 2
dimension : (32256, 45)
128
DPS KDP 1.0 STAT
finished band : 2
dimension : (32508, 45)
129
VIXM VXZ 1.24 STAT
finished band : 2
dimension : (32760, 45)
130
VIXM VXZ_ 1.24 STAT
finished band : 2
dimension : (33012, 45)
131
SPYG VOOG 0.24 STAT
finished band : 2
dimension : (33264, 45)
132
DGP UGL 0.61 NON-STAT
finished band : 2
dimension : (33516, 45)
133
DBLV TTFS 0.98 NON-STAT
finished band : 2
dimension : (33768, 45)
134
FUTY IDU 0.25 NON-STAT
finished band : 2
dimension : (34020, 45)
135
IAU SGOL 0.98 STAT
finished band : 2
dimension : (34272, 45)
136
GLD IAU 9.99 STAT
finished band : 2
dimension : (34524, 45)
137
MIDU UMDD 0.2 STAT
finished band : 2
dimension : (34776, 45)
138
FNCL VFH 0.58 STAT
finished band : 2
dimension : (35028, 45)
139
FSTA VDC 0.24 STAT
finished band : 2
dimension : (35280, 45)
140
IDU VPU 1.18 NON-STAT
finished band : 2
dimension : (35532, 45)
141
LIN PX 0.98 STAT
finished band : 2
dimension : (35784, 45)
142
MIDZ SMDD

/opt/anaconda3/envs/zipline/lib/python3.5/site-packages/ipykernel/__main__.py:52: RuntimeWarning: invalid value encountered in double_scalars


...

finished band : 2
dimension : (51408, 45)
204
CNR NCS 1.0 STAT
finished band : 2
dimension : (51660, 45)
205
NVLN NVLNF 1.0 STAT
finished band : 2
dimension : (51912, 45)
206
OXSQ TICC 0.76 STAT
finished band : 2
dimension : (52164, 45)
207
AUO AUOTY 1.0 STAT
finished band : 2
dimension : (52416, 45)
208
BBDC TCAP 0.93 STAT
finished band : 2
dimension : (52668, 45)
209
EMES EMESQ 1.0 STAT
finished band : 2
dimension : (52920, 45)
210
EMES EMESZ 1.0 STAT
finished band : 2
dimension : (53172, 45)
211
PHIO RXII 55.0 STAT
finished band : 2
dimension : (53424, 45)
212
PTIE SAVA 1.0 STAT
finished band : 2
dimension : (53676, 45)
213
RAS RASFD 1.0 STAT
finished band : 2
dimension : (53928, 45)
214
CBS VIAC 1.01 NON-STAT
finished band : 2
dimension : (54180, 45)
215
CHI_2 QQQC 1.03 STAT
finished band : 2
dimension : (54432, 45)
216
CUR SNCA 1.0 STAT
finished band : 2
dimension : (54684, 45)
217
CWEN NYLD 0.93 STAT
finished band : 2
dimension : (54936, 45)
218
EVLV IMBI 0.1 STAT
finished band :

finished band : 2
dimension : (22932, 45)
91
ITOT IWB 0.41 STAT
finished band : 2
dimension : (23184, 45)
92
DGL DGP 0.77 STAT
finished band : 2
dimension : (23436, 45)
93
SPTL TLT 0.27 NON-STAT
finished band : 2
dimension : (23688, 45)
94
IWB VV 1.19 STAT
finished band : 2
dimension : (23940, 45)
95
ITOT VV 0.49 STAT
finished band : 2
dimension : (24192, 45)
96
DBS SIVR 1.78 STAT
finished band : 2
dimension : (24444, 45)
97
IVW SPYG 4.58 STAT
finished band : 2
dimension : (24696, 45)
98
SPXL SSO 0.45 STAT
finished band : 2
dimension : (24948, 45)
99
ITOT VONE 0.49 STAT
finished band : 2
dimension : (25200, 45)
100
SDS SH 4.44 NON-STAT
finished band : 2
dimension : (25452, 45)
101
VIXY VXX 0.84 STAT
finished band : 2
dimension : (25704, 45)
102
VIXY VXX_ 0.84 STAT
finished band : 2
dimension : (25956, 45)
103
IEV VGK 0.81 STAT
finished band : 2
dimension : (26208, 45)
104
VIIX VXX 2.11 STAT
finished band : 2
dimension : (26460, 45)
105
VIIX VXX_ 2.11 STAT
finished band : 2
dimension : 

216
CUR SNCA 1.0 STAT
finished band : 2
dimension : (54684, 45)
217
CWEN NYLD 0.93 STAT
finished band : 2
dimension : (54936, 45)
218
EVLV IMBI 0.1 STAT
finished band : 2
dimension : (55188, 45)
219
IPCI IPCIF 1.0 STAT
finished band : 2
dimension : (55440, 45)
220
KEG KEGX 1.0 STAT
finished band : 2
dimension : (55692, 45)
221
KEG KEGXD 1.0 STAT
finished band : 2
dimension : (55944, 45)
222
MPVD MPVDF 1.01 STAT
finished band : 2
dimension : (56196, 45)
223
PIPR PJC 0.98 STAT
finished band : 2
dimension : (56448, 45)
224
SDRL SDRL_ 267.77 STAT
finished band : 2
dimension : (56700, 45)
225
SPN SPNV 10.0 STAT
finished band : 2
dimension : (56952, 45)
226
SPN SPNVD 10.0 STAT
finished band : 2
dimension : (57204, 45)
227
TTS TTSH 0.98 STAT
finished band : 2
dimension : (57456, 45)
228
ANW ANWWQ 1.0 STAT
finished band : 2
dimension : (57708, 45)
229
ATCO SSW 1.0 STAT
finished band : 2
dimension : (57960, 45)
230
BAS BASX 1.0 STAT
finished band : 2
dimension : (58212, 45)
231
FEU SPEU 1.01 ST

finished band : 2
dimension : (26208, 45)
104
VIIX VXX 2.11 STAT
finished band : 2
dimension : (26460, 45)
105
VIIX VXX_ 2.11 STAT
finished band : 2
dimension : (26712, 45)
106
BBAR BFR 1.0 STAT
finished band : 2
dimension : (26964, 45)
107
ITOT IYY 0.46 STAT
finished band : 2
dimension : (27216, 45)
108
ZBH ZMH 0.98 NON-STAT
finished band : 2
dimension : (27468, 45)
109
IWB VONE 1.21 STAT
finished band : 2
dimension : (27720, 45)
110
SSO UPRO 2.09 STAT
finished band : 2
dimension : (27972, 45)
111
QQEW QQQE 1.35 STAT
finished band : 2
dimension : (28224, 45)
112
IVW VOOG 1.11 STAT
finished band : 2
dimension : (28476, 45)
113
DBS SLV 1.82 STAT
finished band : 2
dimension : (28728, 45)
114
IWF VONG 0.97 STAT
finished band : 2
dimension : (28980, 45)
115
IWD VONV 1.14 STAT
finished band : 2
dimension : (29232, 45)
116
VONE VV 0.99 STAT
finished band : 2
dimension : (29484, 45)
117
VIA VIAB 1.01 STAT
finished band : 2
dimension : (29736, 45)
118
MGC SPY 0.34 NON-STAT
finished band : 2
di

229
ATCO SSW 1.0 STAT
finished band : 2
dimension : (57960, 45)
230
BAS BASX 1.0 STAT
finished band : 2
dimension : (58212, 45)
231
FEU SPEU 1.01 STAT
finished band : 2
dimension : (58464, 45)
232
FRGA TAX 1.0 STAT
finished band : 2
dimension : (58716, 45)
233
GOV OPI 0.25 STAT
finished band : 2
dimension : (58968, 45)
234
HPT SVC 1.0 STAT
finished band : 2
dimension : (59220, 45)
235
OZM SCU 1.02 NON-STAT
finished band : 2
dimension : (59472, 45)
236
PXMV XMVM 1.02 STAT
finished band : 2
dimension : (59724, 45)
237
RC SLD 0.88 STAT
finished band : 2
dimension : (59976, 45)
238
TAX TAXA 1.0 STAT
finished band : 2
dimension : (60228, 45)
THE END
